In [28]:
# Path to the CORD-19 dataset (adjust as needed based on the dataset structure)
source_folder = '/kaggle/input/CORD-19-research-challenge/document_parses/pdf_json'

# This uploads your service account JSON to the notebook
# You'll need to add this file as a dataset in Kaggle
# See instructions below on how to do this
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../input/google-cloud/adc.json'
bucket_name = "sci_papers_bootcamp"
project_name="jakub-le-wagon-bootcamp"
prefix = "cord19/"

In [20]:
# Install gsutil in the Kaggle notebook
!pip install gsutil
!pip install google-cloud-storage

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 4.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.17.0
    Uninstalling google-auth-2.17.0:
      Successfully uninstalled google-auth-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcs-oauth2-boto-plugin 3.2 requires google-auth==2.17.0, but you have google-auth 2.38.0 which is incompatible.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.38.0 which is incompatible.
google-colab 1.0.0 requires notebook==6.5.5, but you have notebook 6.5.4 which is incompatible.
google-colab 1

In [21]:
from google.cloud.storage import transfer_manager

# Get list of local files
files = glob.glob(f"{source_folder}/*.json", recursive=True)
total_files = len(files)

print(f"Found {total_files} total local files")


Found 401214 total local files


In [40]:
from pathlib import Path

def upload_files_to_gcs(source_directory, project, bucket, prefix):
    """
    Uses Transfer manager to upload files in parallel to a GCS bucket.
    Args:
     source_directory (str): The local path of the source directory to upload files from.
    
    Returns:
    """
    try:   
        client = storage.Client(project=project)
        bucket = client.bucket(bucket)
        if not source_directory:
            print("No local directory provided")
            return

        # Get list of files already in the bucket
        print("Listing existing files in the bucket...")
        existing_files = set()
        blobs = bucket.list_blobs(prefix=prefix)
        for blob in blobs:
            # Remove the prefix to get just the relative path
            if blob.name.startswith(prefix):
                existing_files.add(blob.name[len(prefix):])
        print(f"Found {len(existing_files)} files already in the bucket")
        
        """
        directory_as_path_obj = Path(source_directory)
        paths = directory_as_path_obj.glob('*')
        # Get the list of file paths, relative to the source directory and store them as strings
        file_paths = [path for path in paths if path.is_file()]
        relative_paths = [path.relative_to(source_directory) for path in file_paths]
        string_paths = [str(path) for path in relative_paths]
        """
        # Get list of local files
        files = glob.glob(f"{source_folder}/*.json", recursive=True)
        total_files = len(files)
        
        print(f"Found {total_files} total local files")
        print(f"Example: {files[0]}")

        to_upload = []
        # Create manifest only for files that don't exist in the bucket
        skipped = 0
        for file_path in files:
            relative_path = os.path.relpath(file_path, source_folder)
            if relative_path not in existing_files:
                to_upload.append(relative_path)
            else:
                skipped += 1
    
        print(f"Initiating upload of {len(to_upload)} local files")
        print(f"Starting with: {to_upload[0]}")
        results = transfer_manager.upload_many_from_filenames(
            bucket = bucket,
            filenames = to_upload,
            source_directory = source_directory,
            blob_name_prefix = prefix,
            max_workers = 32,
            skip_if_exists = True,
        )
    except Exception as e:
        print(f"Failed to upload files to GCS due to exception: {e}")

In [41]:
upload_files_to_gcs(source_folder, project_name, bucket_name, prefix)

Listing existing files in the bucket...
Found 2547 files already in the bucket
Found 401214 total local files
Example: /kaggle/input/CORD-19-research-challenge/document_parses/pdf_json/8f97e16f3842e4bbd2d5d1c0c95ac1e31993ec68.json
Initiating upload of 398667 local files
Starting with: d1b3822f36c739fbd34f63ddc02f845e85e921b2.json
